In [13]:
from qdrant_client import QdrantClient, models

In [14]:
client = QdrantClient(url="http://localhost:6333")

In [15]:
if client.collection_exists(collection_name="collection0"):
    client.delete_collection(collection_name="collection0")

client.create_collection(
    collection_name="collection0",
    vectors_config=models.VectorParams(size=3, distance=models.Distance.DOT),
)

client.upsert(
    collection_name="collection0",
    points=models.Batch(
        ids=[1, 2],
        payloads=[
            {
                "country": {
                    "name": "Germany",
                    "cities": [
                        {
                            "name": "Berlin",
                            "population": 3.7,
                            "sightseeing": ["Brandenburg Gate", "Reichstag"]
                        },
                        {
                            "name": "Munich",
                            "population": 1.5,
                            "sightseeing": ["Marienplatz", "Olympiapark"]
                        }
                    ]
                }
            },
            {
                "country": {
                    "name": "Japan",
                    "cities": [
                        {
                            "name": "Tokyo",
                            "population": 9.3,
                            "sightseeing": ["Tokyo Tower", "Tokyo Skytree"]
                        },
                        {
                            "name": "Osaka",
                            "population": 2.7,
                            "sightseeing": ["Osaka Castle", "Universal Studios Japan"]
                        }
                    ]
                }
            }
        ],
        vectors=[
            [0.1, 0.1, 0.1],
            [0.2, 0.2, 0.2],
        ],
    ),
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [16]:
client.scroll(
    collection_name="collection0",
    scroll_filter=models.Filter(
        should=[
            models.FieldCondition(
                # You can search on a nested field using a dot notation.
                key="country.name", match=models.MatchValue(value="Germany")
            ),
        ],
    ),
)

([Record(id=1, payload={'country': {'cities': [{'name': 'Berlin', 'population': 3.7, 'sightseeing': ['Brandenburg Gate', 'Reichstag']}, {'name': 'Munich', 'population': 1.5, 'sightseeing': ['Marienplatz', 'Olympiapark']}], 'name': 'Germany'}}, vector=None, shard_key=None, order_value=None)],
 None)

In [17]:
client.scroll(
    collection_name="collection0",
    scroll_filter=models.Filter(
        should=[
            models.FieldCondition(
                # You can also search through arrays by projecting inner values using the [] syntax.
                key="country.cities[].population",
                range=models.Range(
                    gt=None,
                    gte=9.0,
                    lt=None,
                    lte=None,
                ),
            ),
        ],
    ),
)

([Record(id=2, payload={'country': {'cities': [{'name': 'Tokyo', 'population': 9.3, 'sightseeing': ['Tokyo Tower', 'Tokyo Skytree']}, {'name': 'Osaka', 'population': 2.7, 'sightseeing': ['Osaka Castle', 'Universal Studios Japan']}], 'name': 'Japan'}}, vector=None, shard_key=None, order_value=None)],
 None)

In [18]:
client.scroll(
    collection_name="collection0",
    scroll_filter=models.Filter(
        should=[
            models.FieldCondition(
                key="country.cities[].sightseeing",
                match=models.MatchValue(value="Osaka Castle"),
            ),
        ],
    ),
)

([Record(id=2, payload={'country': {'cities': [{'name': 'Tokyo', 'population': 9.3, 'sightseeing': ['Tokyo Tower', 'Tokyo Skytree']}, {'name': 'Osaka', 'population': 2.7, 'sightseeing': ['Osaka Castle', 'Universal Studios Japan']}], 'name': 'Japan'}}, vector=None, shard_key=None, order_value=None)],
 None)